In [ ]:
import pandas as pd
data = {'Name': ['Alice', 'Bob', 'Charlie'],
        'Age': [25, 30, 35]}
df = pd.DataFrame(data)
print(df)
new_data = {'Name': ['David', 'Eve'],
            'Age': [28, 22]}

new_df = pd.DataFrame(new_data)
print(new_df)
# Append new data to the existing DataFrame
df = pd.concat([df, new_df], ignore_index=True)

#records = df.to_dict(orient='records')
df.to_json('output.json', orient='records', lines=True)

In [ ]:
f = './MLM/mlm_prepared_data/dev_mlm_abolish_full.json'
json_data = pd.read_json(f, lines=True)
#train, dev, test = np.split(df.sample(frac=1), [int(.6*len(df)), int(.8*len(df))])
train, testt = train_test_split(json_data, test_size=0.4)
dev, test = train_test_split(testt, test_size=0.5)

In [ ]:
print(type(train))

In [ ]:
df = pd.concat([train, test], ignore_index=True)
df.to_json('output.json', orient='records', lines=True)

In [ ]:
token_id = [101,1748,4982,117,5190,4789,1107,19255,118,4065,3652,1144,1151,2423,8632,1118,1103,3687,26883,1320,1104,194,3031,1162,117,8783,1115,194,3031,1162,2399,170,3607,1648,1107,19255,118,4065,5190,4789,119,102,0,0,0,0,0,0,0,0]
mask = [1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0]
# {"uid":10,"token_id":[101,1748,4982,117,5190,4789,1107,19255,118,4065,3652,1144,1151,2423,8632,1118,1103,3687,26883,1320,1104,194,3031,1162,117,8783,1115,194,3031,1162,2399,170,3607,1648,1107,19255,118,4065,5190,4789,119,102,0,0,0,0,0,0,0,0],"mask":[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0],"pos":["ADV","PUNCT","NOUN","NOUN","ADP","ADJ","PUNCT","NOUN","NOUN","AUX","AUX","ADV","VERB","ADP","DET","NOUN","ADP","NOUN","PUNCT","VERB","SCONJ","PROPN","VERB","DET","ADJ","NOUN","ADP","ADJ","PUNCT","NOUN","NOUN","NOUN","PUNCT"]}

In [16]:
import joblib

res = joblib.load('./data/coNLL_tsv/ner_coNLL_train2_label_map.joblib')
res

{'B-S_NP_NP_NN': 0,
 'B-NN': 1,
 'O-NP': 2,
 'B-PP_IN': 3,
 'B-NP_DT': 4,
 'O-PP': 5,
 'B-VP_VBZ': 6,
 'B-VP_VBN': 7,
 'B-NP_PP_IN': 8,
 'B-NP_NP_NN': 9,
 'O-VP': 10,
 'B-PERIOD': 11,
 'O-S': 12,
 'B-S_NP_NN': 13,
 'B-NNS': 14,
 'B-VP_VBP': 15,
 'B-NP_NP_JJ': 16,
 'B-SBAR_WHNP_WDT': 17,
 'O-WHNP': 18,
 'B-S_NP_VP_VBP': 19,
 'B-VBG': 20,
 'B-SBAR_WHNP_S_NP_VP_O-SBAR': 21,
 'O-SBAR': 22,
 'B-S_NP_NP_DT': 23,
 'B-JJ': 24,
 'B-NP_NN': 25,
 'B-RB': 26,
 'B-VP_VB': 27,
 'B-NP_NP_NNS': 28,
 'B-COMMA': 29,
 'B-SBAR_IN': 30,
 'B-NP_ADVP_FW': 31,
 'B-FW': 32,
 'O-ADVP': 33,
 'B-NP_JJ': 34,
 'B-S_ADVP_RB': 35,
 'B-NP_NP_DT': 36,
 'B-S_PP_IN': 37,
 'B-NP_NP_PP_O-PP': 38,
 'B-NP_VP_VBD': 39,
 'B-S_NP_VP_TO': 40,
 'B-CC': 41,
 'B-PRN_LRB': 42,
 'B-NP_NNS': 43,
 'B-RRB': 44,
 'O-PRN': 45,
 'B-VP_VBD': 46,
 'B-NP_O-VP': 47,
 'B-S_S_NP_DT': 48,
 'B-ADJP_ADJP_RB': 49,
 'B-JJR': 50,
 'O-ADJP': 51,
 'B-PP_O-ADJP': 52,
 'B-COLON': 53,
 'B-NP_NP_CD': 54,
 'B-S_NP_NP_NP_DT': 55,
 'B-S_NP_VP_VBG': 56,
 'B-S_S

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
f = open('./MLM/mlm_output/mlm_abolish_full.json')
json_data = pd.read_json(f, lines=True)
df = pd.read_json(f, lines=True)
train, dev, test = np.split(df.sample(frac=1), [int(.6*len(df)), int(.8*len(df))])


In [ ]:
print(type(train))

In [ ]:
saved_train = {}
print(train)
saved_train.update(train)


In [ ]:
# write to json
import json
with open('train.json', 'w') as outfile:
    json.dump(saved_train, outfile)